### PART A: Frequent Itemset Generation

In [1]:
# Importing the libraries and the dataset

import numpy as np 
import pandas as pd

Market_Data = pd.read_csv('Market_Basket_Optimisation.csv',index_col=None, header = None ) # Use your local path here

Market_Data.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,whole wheat pasta,french fries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,soup,light cream,shallot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,frozen vegetables,spaghetti,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,french fries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Converting the Market Dataset into a nested list
dataSet = []

for index, transaction in Market_Data.iterrows():
    cleaned_transaction = transaction[~transaction.isnull()].tolist()
    dataSet.append(cleaned_transaction)

#### 1. The createItem function

In [3]:
# For the given dataset writing a function to return the list of distinct items in the dataset

def createItem(dataSet):
    
    itemList = []
    
    for transaction in dataSet:
        for item in transaction:
            if not [item] in itemList:
                
                 # creating unique single lists in Itemlist. ie list of all items
                itemList.append([item])
                
    itemList.sort()
    
    return list(map(frozenset, itemList))

#### 2. The scanData function

In [4]:
# Returns an Itemset and dictionary with Support values

def scanData(data, itemsetk, minSupport):
    
    tempDict = {}
    
    for transaction in data:
        for item in itemsetk:
            if item.issubset(transaction):
                if not item in tempDict: 
                    tempDict[item]=1 # tempDict contains number of all items
                else: 
                    tempDict[item] += 1
    
    numItems = float(len(data))
    freqItemset = []
    supportDict = {}
    
    for key in tempDict:
        support = tempDict[key]/numItems 
        
        if support >= minSupport:
            freqItemset.insert(0,key) # freqItemset contains all frequent items
        supportDict[key] = support # contains support of all items
    
    return freqItemset, supportDict

#### 3. The itemSetGenerator function

In [5]:
# Creating Higher order Itemsets

def itemSetGenerator(itemsetk, k):
 
    higherOrderitemset = []
    lenitemsetk = len(itemsetk)
    
    
    for i in range(lenitemsetk):
        for j in range(i+1, lenitemsetk): 
            L1 = sorted(list(itemsetk[i]))[:k-2] 
            L2 = sorted(list(itemsetk[j]))[:k-2] 
            
            # Two frequent itemsets of order k are merged only if their k-1 itemsets are identical
            if L1 == L2:
                higherOrderitemset.append(itemsetk[i] | itemsetk[j]) # Performing set union creates itemset with n+1 items
               
    return higherOrderitemset

#### 4. Frequent Itemsets Generation

In [6]:
def apriori(dataSet, minSupport):
    
    itemList = createItem(dataSet) # Creating frozenset of items
    
    
    # Generating all the frequent 1-itemsets and the support of those items
    freqItemset1, supportDict = scanData(dataSet, itemList, minSupport)
    freqItemsets = [freqItemset1]
    k = 2 
    
    while (len(freqItemsets[k-2]) > 0): # Incrementing k until we no longer find any kth order itemsets
        
        itemsetk = itemSetGenerator(freqItemsets[k-2], k) # Generating itemsets of order k
        
        # Generating the frequent itemset for the kth order and support for each of these itemsets
        freqItemsetk, supportDictk = scanData(dataSet, itemsetk, minSupport) 
        
        supportDict.update(supportDictk)
        freqItemsets.append(freqItemsetk)
        
        k += 1
    return freqItemsets, supportDict

Calculate the maximum possible two itemsets.
- 140
- 120
- 7140
- 5240

In [7]:
# Identify the distinct items involved in the dataset

"""
To identify the maximum possible two itemsets, the first step would be to identify 
the total number of items in the given dataset.
"""

Items = createItem(dataSet)
len(Items)

120

In [8]:
# Calculate the number of possible two itemsets
"""
Next step is to calculate nCr since you need all the possible two-item combinations. 
Here n is the total number of distinct items

120C2 = 7140 

Note: Instead of calculating this manually, you can write a function to 
calculate nCr for given values of n and r.
"""
import operator as op
from functools import reduce

def ncr(n, r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer // denom

print(ncr(120, 2))

7140


 Identify the support of itemset {'eggs','mineral water','spaghetti'} [ Round the value upto three decimal points]

- 0.022
- 0.034
- 0.014
- 0.052

Identify the number of transaction where items eggs, mineral water and spaghetti are bought together

- 165
- 255
- 392
- 107

In [10]:
# Find the support of the itemset {'eggs', 'mineral water', 'spaghetti'}

"""
In the frequent itemset generation phase, a support dictionary is created 
with itemset as the key and support as the value. You can find the 
support value of the given itemset using this support dictionary.

"""

freqItemsets, supportDict = apriori(dataSet, 0.001)
x = frozenset({'eggs', 'mineral water', 'spaghetti'})
supportX = supportDict[x]
print(f"Support for itemset {{'eggs', 'mineral water', 'spaghetti'}} = {supportX}")

Support for itemset {'eggs', 'mineral water', 'spaghetti'} = 0.014264764698040262


In [11]:
# Identify the total number of Transactions in the dataset

lengthTxn = len(dataSet)
print(f"Total number of transaction for the itemset {{'eggs', 'mineral water', 'spaghetti'}} = {supportX * lengthTxn}")


Total number of transaction for the itemset {'eggs', 'mineral water', 'spaghetti'} = 107.0


Identify the number of frequent itemsets involving both chocolate and mineral water if minSupport is defined as 0.001

- 244
- 502
- 343
- 156

In [12]:
# Generate the frequent itemsets when minSupport = 0.001

"""
First, you generate all the frequent itemsets.
"""

freqItemsets, supportDict= apriori(dataSet, minSupport=0.001)


In [13]:
# Write a code to find the number of frequent itemset containing both chocolate and mineral water

"""
Then you write a function to check if a particular frequent 
itemset contains the items chocolate and mineral water.
"""

x = frozenset({'chocolate', 'mineral water'})
count = 0

for itemsetsK in freqItemsets:
    for items in itemsetsK:
        if items.intersection(x) == frozenset({'chocolate', 'mineral water'}) :
            count = count +1
                   
print(count)


245


### PART B: Rule Generation

#### 5. The ‘calcConf’ function

In [14]:
def calcConf(freqSet, H, supportDict, bigRuleList, minConf, minLift):

    prunedH = []
    
    for conseq in H:
        # calculate confidence
        conf = supportDict[freqSet]/supportDict[freqSet - conseq] 
        
        # calculate lift
        lift = supportDict[freqSet]/(supportDict[freqSet - conseq]*supportDict[conseq])
        
        if conf >= minConf and lift >= minLift:
            bigRuleList.append((freqSet-conseq, conseq, conf, lift))
            print(freqSet-conseq, '--->', conseq, '; confidence = ', "{:.5f}".format(conf), ', lift = ', "{:.5f}".format(lift))
            prunedH.append(conseq)

    return prunedH

#### 6. The rulesFromConseq function


In [15]:
def rulesFromConseq(freqSet, H, supportDict, bigRuleList, minConf, minLift):

    m = len(H[0]) # Order of the consequent while generating the rules
         
    H = calcConf(freqSet, H, supportDict, bigRuleList, minConf, minLift)
    if len(H)>1: # For len(H)<=1, you cannot generate higher order cadnidates
        
        # creating higher order candidates
        Hmp1 = itemSetGenerator(H, m+1) 
        
        if Hmp1 == []: # This will happen if higher order consequent itemsets are not possible
            # Hmp1 will be an empty list if the itemsets in H don't satisfy the condition for merging
            return 0
        
        if (len(Hmp1[0]) < len(freqSet)):
            # Generate rules while the order of the itemsets in Hmp1 is less than the number of items in the frequent itemset
            rulesFromConseq(freqSet, Hmp1, supportDict, bigRuleList, minConf, minLift)

#### 7. The generateRules function


In [16]:
def generateRules(freqItemsets, supportDict, minConf, minLift):  #supportDict is a dictionary coming from scanData
    bigRuleList = []
    for i in range(1, len(freqItemsets)): # Only get the sets with two or more items
        for freqSet in freqItemsets[i]:
            H1 = [frozenset([item]) for item in freqSet]  
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportDict, bigRuleList, minConf, minLift)
            else:
                calcConf(freqSet, H1, supportDict, bigRuleList, minConf, minLift)
    return bigRuleList 

In [17]:
# In the code for generating rules, implement the computation of Lift


"""
In the code to generate rules, you need to add a new metric lift. 
For this, you can do either of the two things; You can modify the code 
such that each method takes metric lift as an input parameter specified 
by the user. or During the computation of confidence, you can add the 
computation for lift. Then you can generate the rules append rules to 
the final list only when the lift is greater than the specified value

"""


minSupport=0.05


freqItemsets, supportDict= apriori(dataSet, minSupport)



In [18]:
# Generate the rules when minSupport = 0.05, minConfidence = 0.2 and lift >1.2 
minConfidence = 0.2
minLift = 1.2
rules= generateRules(freqItemsets, supportDict, minConfidence, minLift)


frozenset({'mineral water'}) ---> frozenset({'chocolate'}) ; confidence =  0.22092 , lift =  1.34833
frozenset({'chocolate'}) ---> frozenset({'mineral water'}) ; confidence =  0.32140 , lift =  1.34833
frozenset({'mineral water'}) ---> frozenset({'spaghetti'}) ; confidence =  0.25056 , lift =  1.43909
frozenset({'spaghetti'}) ---> frozenset({'mineral water'}) ; confidence =  0.34303 , lift =  1.43909


In [19]:
# count the number of rules

len(rules)

4

Imagine that for some threshold support value, the itemset  {'eggs','chocolate','spaghetti'} is found to be a frequent itemset. Now for the rule {'chocolate'} --> {'spaghetti', 'eggs'} to be valid what can be the max value of minConfidence? [More than one answer type question]

- 0.08
- 0.04
- 0.06
- 0.01

In [20]:
#Itendify the support of {'eggs', 'mineral water', 'chocolate'}




In [21]:
# What will be the confidence of the rule {'chocolate'} --> {'spaghetti', 'eggs'}




It is given that “for some threshold support, the itemset 
{'eggs','chocolate', 'spaghetti'} is found to be frequent”.
For this statement to be valid you need to assume a minSupport 
such that this will be part of the frequent itemsets. 

Using the support dictionary identify the supports of  itemsets 
{'chocolate'} and {'eggs','chocolate','spaghetti'}. 

Using these two values you can find the confidence of the rule 
{'chocolate'} --> {'spaghetti', 'eggs'}. 

Finally, for this rule to be valid, the value of minimum Confidence should be 
less than the calculated value.


Ans: 0.01, 0.04, 0.06
